In [1]:
import collections
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
%config InlineBackend.figure_format='retina'

In [3]:
orig_data = pd.read_csv('../AllEpitopeFeatures.csv')
orig_data = orig_data.rename(columns={'Epitope': 'sequence'})
orig_data['sequence_length'] = [len(x) for x in orig_data['sequence'].values]
orig_data

,protein,sequence,entropy_sum,epi_start_pos,epi_len,glyco_probs,crosses_cleavage,sequence_length
0,E,MYSFVSEE,0.002395,0,8,0.0,0,8
1,E,YSFVSEET,0.002395,1,8,0.0,0,8
2,E,SFVSEETG,0.002395,2,8,0.0,0,8
3,E,FVSEETGT,0.002395,3,8,0.0,0,8
4,E,VSEETGTL,0.002395,4,8,0.0,0,8
...,...,...,...,...,...,...,...,...
174523,S2,KGCCSCGSCCKFDEDDSEPVLKGVK,0.095264,1244,25,0.0,0,25
174524,S2,GCCSCGSCCKFDEDDSEPVLKGVKL,0.092869,1245,25,0.0,0,25
174525,S2,CCSCGSCCKFDEDDSEPVLKGVKLH,0.092869,1246,25,0.0,0,25
174526,S2,CSCGSCCKFDEDDSEPVLKGVKLHY,0.086502,1247,25,0.0,0,25


In [4]:
# Load final set of HLA alleles.
hla_alleles = pd.read_csv('MHC2_allele_marry.txt', names=['allele'])
hla_alleles

,allele
0,HLA-DPA10301-DPB11301
1,HLA-DPA10201-DPB155801
2,DRB1_0701
3,HLA-DPA10207-DPB116201
4,HLA-DPA10301-DPB15501
...,...
313,HLA-DQA10401-DQB10301
314,HLA-DPA10103-DPB112601
315,HLA-DPA10301-DPB16501
316,HLA-DPA10202-DPB11301


In [8]:
# Filter MHC-II sequences to epitopes with sequence length 13-25 (inclusive).
mhc2_data = orig_data.loc[orig_data['sequence_length'].isin(range(13,26))]
mhc2_data

,protein,sequence,entropy_sum,epi_start_pos,epi_len,glyco_probs,crosses_cleavage,sequence_length
330,E,MYSFVSEETGTLI,0.002395,0,13,0.0,0,13
331,E,YSFVSEETGTLIV,0.002395,1,13,0.0,0,13
332,E,SFVSEETGTLIVN,0.002395,2,13,0.0,0,13
333,E,FVSEETGTLIVNS,0.002395,3,13,0.0,0,13
334,E,VSEETGTLIVNSV,0.004789,4,13,0.0,0,13
...,...,...,...,...,...,...,...,...
174523,S2,KGCCSCGSCCKFDEDDSEPVLKGVK,0.095264,1244,25,0.0,0,25
174524,S2,GCCSCGSCCKFDEDDSEPVLKGVKL,0.092869,1245,25,0.0,0,25
174525,S2,CCSCGSCCKFDEDDSEPVLKGVKLH,0.092869,1246,25,0.0,0,25
174526,S2,CSCGSCCKFDEDDSEPVLKGVKLHY,0.086502,1247,25,0.0,0,25


In [7]:
# Write peptides out (unpaired with MHC) for NetMHCpan.
mhc2_data[['sequence']].to_csv('peptides_13-25.pep', index=False, header=False)

# Create commands for running NetMHCIIpan-3.2 and NetMHCIIpan-4.0

In [9]:
# Create commands for running NetMHCIIpan3.2 (MHC-II).
cmd_template = '-inptype 1 -f peptides_13-25.pep -a {allele} -xls -xlsfile {allele_file}'
cmds = []
for allele in hla_alleles['allele'].values:
    cmd = cmd_template.format(
        allele=allele.replace('*', '_').replace(':', ''),
        allele_file='netmhcii-3.2_preds/%s_preds.xls' % (allele.replace('*', '_').replace(':', ''))
    )
    cmds.append(cmd)
print('# cmds: ', len(cmds))
with open('netmhc_class2_args.txt', 'w') as f:
    for cmd in cmds:
        f.write(cmd+'\n')

# cmds:  318


In [10]:
# Create commands for running NetMHCIIpan4.0 (MHC-II).
cmd_template = '-inptype 1 -f peptides_13-25.pep -a {allele} -BA -xls -xlsfile {allele_file}'
cmds = []
for allele in hla_alleles['allele'].values:
    cmd = cmd_template.format(
        allele=allele.replace('*', '_').replace(':', ''),
        allele_file='netmhcii-4.0_preds/%s_preds.xls' % (allele.replace('*', '_').replace(':', ''))
    )
    cmds.append(cmd)
print('# cmds: ', len(cmds))
with open('netmhc4.0_class2_args.txt', 'w') as f:
    for cmd in cmds:
        f.write(cmd+'\n')

# cmds:  318


Command to run:

```
cat netmhc_class2_args.txt | xargs -P 70 -d '\n' -n 1 ./netMHCIIpan-3.2/netMHCIIpan

cat netmhc4.0_class2_args.txt | xargs -P 70 -d '\n' -n 1 ./netMHCIIpan-4.0/netMHCIIpan
``````

# Load NetMHCIIpan4.0 predictions

In [17]:
dfs = []
for allele in hla_alleles['allele'].values:
    try:
        df = pd.read_csv(
            './netmhcii-4.0_preds/%s_preds.xls' % allele.replace(':', ''),
            delimiter='\t',
            skiprows=[0],
        )
    except:
        continue
    df['genotype'] = allele
    df = df.drop(columns=['Pos', 'ID', 'Ave', 'NB'])
    dfs.append(df)
print('Loaded %d alleles' % len(dfs))
netmhc2_data = pd.concat(dfs)
netmhc2_data['sequence_length'] = [len(x) for x in netmhc2_data['Peptide'].values]
netmhc2_data['loci'] = [x[:4] if x[:3] == 'DRB' else x[:6] for x in netmhc2_data['genotype'].values]
# netmhc2_data['1-log50k'] = 1 - np.log(netmhc2_data['nM']) / np.log(50000)
netmhc2_data

Loaded 280 alleles


,Peptide,Target,Score,Rank,Score_BA,nM,Rank_BA,genotype,sequence_length,loci
0,MYSFVSEETGTLI,-99.999,0.041539,35.55,0.246930,3456.633552,50.21,HLA-DPA10301-DPB11301,13,HLA-DP
1,YSFVSEETGTLIV,-99.999,0.038426,39.10,0.240716,3697.028008,52.48,HLA-DPA10301-DPB11301,13,HLA-DP
2,SFVSEETGTLIVN,-99.999,0.028666,53.15,0.212631,5009.833741,62.80,HLA-DPA10301-DPB11301,13,HLA-DP
3,FVSEETGTLIVNS,-99.999,0.036350,41.72,0.206312,5364.337635,65.16,HLA-DPA10301-DPB11301,13,HLA-DP
4,VSEETGTLIVNSV,-99.999,0.043983,32.96,0.200067,5739.329682,67.36,HLA-DPA10301-DPB11301,13,HLA-DP
...,...,...,...,...,...,...,...,...,...,...
125588,KGCCSCGSCCKFDEDDSEPVLKGVK,-99.999,0.002639,91.75,0.168159,8105.800438,79.10,HLA-DPA10103-DPB14101,25,HLA-DP
125589,GCCSCGSCCKFDEDDSEPVLKGVKL,-99.999,0.003358,88.46,0.173627,7640.150953,77.25,HLA-DPA10103-DPB14101,25,HLA-DP
125590,CCSCGSCCKFDEDDSEPVLKGVKLH,-99.999,0.003350,88.49,0.174621,7558.422479,76.91,HLA-DPA10103-DPB14101,25,HLA-DP
125591,CSCGSCCKFDEDDSEPVLKGVKLHY,-99.999,0.003275,88.85,0.214361,4916.930790,62.79,HLA-DPA10103-DPB14101,25,HLA-DP


In [7]:
# Write predicted binding affinity predictions.

df = netmhc2_data

df2 = df.groupby(['Peptide', 'loci']).count().reset_index()[['Peptide', 'loci']]
df2['genotype'] = 'unknown'
df2['Score'] = 0.
df2['Rank'] = 0.
df2['Score_BA'] = 0.
df2['Rank_BA'] = 0.
df2['nM'] = 0.

data_pivot = pd.concat([df, df2], sort=False).pivot_table(
    index='Peptide',
    columns=['loci', 'genotype'],
    values='Score_BA',
)
data_pivot.to_pickle('mhc2_haplotype_netmhcii-4.0_pred_affinity_pivot_v1v2.pkl.gz', protocol=2)
data_pivot

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT              0.262275  0.210513  0.162170  0.138032  0.064328   
AAAYYVGYLQPRTF             0.435325  0.357479  0.232249  0.188956  0.081843   
AAAYYVGYLQPRTFL            0.620743  0.533288  0.350694  0.281340  0.119930   
AAAYYVGYLQPRTFLL           0.603934  0.471715  0.317015  0.240276  0.107944   
AAAYYVGYLQPRTFLLK          0.669816  0.519657  0.374913  0.280090  0.120530   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM      0.399611  0.273032  0.203689  0.191331  0.094848   
YYVWKSYVHVVDGCNSSTCMMC     0.400962  0.277270  0.206981  0.195065  0.095416   
YYVWKSYVHVVDGCNSSTCMMCY    0.402886  0.282106  0.209602  0.198191  0.096931   
YYVWKSYVHVVDGCNSSTCMMCYK   0.405321  0.287790  0.211981  0.201682  0.097617   
YYVWKSYVHVVDGCNSSTCMMCYKR  0.409240  0.294411  0.214682  0.205183  0.098597   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT              0.200257  0.182535  0.152988  0.196000  0.237120   
AAAYYVGYLQPRTF             0.289715  0.248274  0.208763  0.302934  0.341891   
AAAYYVGYLQPRTFL            0.388640  0.326797  0.272157  0.420422  0.449729   
AAAYYVGYLQPRTFLL           0.385170  0.287178  0.258092  0.384697  0.441300   
AAAYYVGYLQPRTFLLK          0.433591  0.309963  0.284692  0.428667  0.472981   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM      0.360642  0.217532  0.232871  0.344261  0.421699   
YYVWKSYVHVVDGCNSSTCMMC     0.363595  0.220804  0.233858  0.347686  0.422885   
YYVWKSYVHVVDGCNSSTCMMCY    0.365776  0.222661  0.234946  0.350836  0.424360   
YYVWKSYVHVVDGCNSSTCMMCYK   0.369570  0.224174  0.236694  0.353541  0.425438   
YYVWKSYVHVVDGCNSSTCMMCYKR  0.372695  0.226051  0.238848  0.356386  0.426486   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10309 HLA-DQA10505-DQB10319   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...              0.249702              0.249702   
AAAYYVGYLQPRTF             ...              0.333322              0.333322   
AAAYYVGYLQPRTFL            ...              0.383470              0.383470   
AAAYYVGYLQPRTFLL           ...              0.374134              0.374134   
AAAYYVGYLQPRTFLLK          ...              0.354784              0.354784   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...              0.285683              0.285683   
YYVWKSYVHVVDGCNSSTCMMC     ...              0.292342              0.292342   
YYVWKSYVHVVDGCNSSTCMMCY    ...              0.306004              0.306004   
YYVWKSYVHVVDGCNSSTCMMCYK   ...              0.319624              0.319624   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...              0.333877              0.333877   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10402 HLA-DQA10505-DQB10501   
Peptide                                                                 
AAAYYVGYLQPRT                          0.297920              0.264890   
AAAYYVGYLQPRTF                         0.435867              0.339701   
AAAYYVGYLQPRTFL                        0.512765              0.384206   
AAAYYVGYLQPRTFLL                       0.490339              0.372928   
AAAYYVGYLQPRTFLLK                      0.482520              0.379043   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

In [9]:
# Write predicted rank (EL score) predictions.

df = netmhc2_data

data_pivot = netmhc2_data.pivot_table(
    index='Peptide',
    columns=['loci', 'genotype'],
    values='Rank',
)
# Invert rank so threshold is >= 99.5 rather than <= 0.5.
data_pivot = 100 - data_pivot
data_pivot.to_pickle('mhc2_haplotype_netmhcii-4.0_el_rank_pivot_v2.pkl.gz', protocol=2)
data_pivot

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT                 49.55     42.30     58.67     63.13     73.96   
AAAYYVGYLQPRTF                56.91     59.24     69.37     62.94     75.26   
AAAYYVGYLQPRTFL               58.92     73.44     71.54     58.67     73.21   
AAAYYVGYLQPRTFLL              51.01     67.06     64.39     50.77     59.00   
AAAYYVGYLQPRTFLLK             69.09     74.68     74.51     41.10     49.39   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM          5.35      5.00     10.74      5.24      8.20   
YYVWKSYVHVVDGCNSSTCMMC         5.78      5.00     11.53      5.13      8.50   
YYVWKSYVHVVDGCNSSTCMMCY        6.27      5.42     12.38      5.42      8.80   
YYVWKSYVHVVDGCNSSTCMMCYK       6.73      5.82     13.23      5.68      9.00   
YYVWKSYVHVVDGCNSSTCMMCYKR      7.17      6.27     13.79      5.87      9.30   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT                 47.28     53.43     47.67     30.49     45.41   
AAAYYVGYLQPRTF                48.72     68.74     50.66     38.28     55.17   
AAAYYVGYLQPRTFL               40.21     65.65     42.11     29.50     47.17   
AAAYYVGYLQPRTFLL              27.82     53.27     30.22     21.77     37.50   
AAAYYVGYLQPRTFLLK             42.60     46.21     30.74     23.76     33.08   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM         41.29     17.81     25.87     24.18     56.19   
YYVWKSYVHVVDGCNSSTCMMC        42.73     18.43     26.91     25.46     57.03   
YYVWKSYVHVVDGCNSSTCMMCY       43.94     19.29     27.82     26.36     57.68   
YYVWKSYVHVVDGCNSSTCMMCYK      44.96     20.01     28.60     27.12     58.19   
YYVWKSYVHVVDGCNSSTCMMCYKR     45.62     20.51     29.26     27.89     58.61   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10302 HLA-DQA10505-DQB10309   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...                 40.69                 84.33   
AAAYYVGYLQPRTF             ...                 50.49                 82.80   
AAAYYVGYLQPRTFL            ...                 41.01                 75.16   
AAAYYVGYLQPRTFLL           ...                 35.00                 64.62   
AAAYYVGYLQPRTFLLK          ...                 31.49                 52.17   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...                  6.48                  5.07   
YYVWKSYVHVVDGCNSSTCMMC     ...                  6.38                  5.03   
YYVWKSYVHVVDGCNSSTCMMCY    ...                  6.83                  5.23   
YYVWKSYVHVVDGCNSSTCMMCYK   ...                  7.11                  5.89   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...                  7.46                  9.21   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10319 HLA-DQA10505-DQB10402   
Peptide                                                                 
AAAYYVGYLQPRT                             84.33                 72.33   
AAAYYVGYLQPRTF                            82.80                 78.55   
AAAYYVGYLQPRTFL                           75.16                 68.35   
AAAYYVGYLQPRTFLL                          64.62                 52.23   
AAAYYVGYLQPRTFLLK                         52.17                 39.41   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

In [9]:
# Write predicted rank (BA score) predictions.

df = netmhc2_data

data_pivot = netmhc2_data.pivot_table(
    index='Peptide',
    columns=['loci', 'genotype'],
    values='Rank_BA',
)
# Invert rank so threshold is >= 99.5 rather than <= 0.5.
data_pivot = 100 - data_pivot
data_pivot.to_pickle('mhc2_haplotype_netmhcii-4.0_ba_rank_pivot_v2.pkl.gz', protocol=2)
data_pivot

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT                 31.27     27.39     36.80     28.28     34.18   
AAAYYVGYLQPRTF                67.76     69.94     67.74     49.59     56.03   
AAAYYVGYLQPRTFL               92.52     95.35     92.74     78.44     84.58   
AAAYYVGYLQPRTFLL              91.12     90.22     88.57     67.90     78.45   
AAAYYVGYLQPRTFLLK             95.70     94.47     94.79     78.17     84.84   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM         60.90     46.33     56.44     50.53     69.03   
YYVWKSYVHVVDGCNSSTCMMC        61.17     47.64     57.82     52.02     69.52   
YYVWKSYVHVVDGCNSSTCMMCY       61.55     49.13     58.92     53.27     70.78   
YYVWKSYVHVVDGCNSSTCMMCYK      62.02     50.81     59.90     54.57     71.32   
YYVWKSYVHVVDGCNSSTCMMCYKR     62.79     52.76     61.01     55.94     72.08   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT                 30.15     42.73     30.35     27.78     40.20   
AAAYYVGYLQPRTF                58.28     69.87     57.24     57.25     69.57   
AAAYYVGYLQPRTFL               82.62     89.66     81.19     83.07     89.92   
AAAYYVGYLQPRTFLL              81.96     81.51     76.86     76.52     88.86   
AAAYYVGYLQPRTFLLK             89.30     86.61     84.49     84.41     92.48   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM         76.94     58.06     67.65     67.61     85.89   
YYVWKSYVHVVDGCNSSTCMMC        77.59     59.38     68.04     68.39     86.08   
YYVWKSYVHVVDGCNSSTCMMCY       78.06     60.10     68.46     69.08     86.32   
YYVWKSYVHVVDGCNSSTCMMCYK      78.88     60.71     69.14     69.75     86.49   
YYVWKSYVHVVDGCNSSTCMMCYKR     79.54     61.45     69.98     70.39     86.65   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10302 HLA-DQA10505-DQB10309   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...                 38.97                 44.30   
AAAYYVGYLQPRTF             ...                 74.25                 69.05   
AAAYYVGYLQPRTFL            ...                 81.16                 79.14   
AAAYYVGYLQPRTFLL           ...                 80.93                 77.50   
AAAYYVGYLQPRTFLLK          ...                 71.93                 73.72   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...                 43.98                 56.32   
YYVWKSYVHVVDGCNSSTCMMC     ...                 45.22                 58.26   
YYVWKSYVHVVDGCNSSTCMMCY    ...                 52.55                 62.05   
YYVWKSYVHVVDGCNSSTCMMCYK   ...                 53.74                 65.68   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...                 55.52                 69.18   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10319 HLA-DQA10505-DQB10402   
Peptide                                                                 
AAAYYVGYLQPRT                             44.30                 51.11   
AAAYYVGYLQPRTF                            69.05                 89.15   
AAAYYVGYLQPRTFL                           79.14                 97.49   
AAAYYVGYLQPRTFLL                          77.50                 95.89   
AAAYYVGYLQPRTFLLK                         73.72                 95.21   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

# Load NetMHCIIpan3.2 predictions

In [6]:
dfs = []
for allele in hla_alleles['allele'].values:
    try:
        df = pd.read_csv(
            './netmhcii-3.2_preds/%s_preds.xls' % allele.replace(':', ''),
            delimiter='\t',
            skiprows=[0],
        )
    except:
        continue
    df['genotype'] = allele
    df = df.drop(columns=['Pos', 'ID', 'Ave', 'NB'])
    dfs.append(df)
print('Loaded %d alleles' % len(dfs))
netmhc2_data = pd.concat(dfs)
netmhc2_data['sequence_length'] = [len(x) for x in netmhc2_data['Peptide'].values]
netmhc2_data['loci'] = [x[:4] if x[:3] == 'DRB' else x[:6] for x in netmhc2_data['genotype'].values]
# netmhc2_data['1-log50k'] = 1 - np.log(netmhc2_data['nM']) / np.log(50000)
netmhc2_data

Loaded 280 alleles


,Peptide,1-log50k,nM,Rank,genotype,sequence_length,loci
0,MYSFVSEETGTLI,0.209,5188.97,46.0,HLA-DPA10301-DPB11301,13,HLA-DP
1,YSFVSEETGTLIV,0.211,5106.81,45.0,HLA-DPA10301-DPB11301,13,HLA-DP
2,SFVSEETGTLIVN,0.189,6500.21,55.0,HLA-DPA10301-DPB11301,13,HLA-DP
3,FVSEETGTLIVNS,0.167,8166.10,65.0,HLA-DPA10301-DPB11301,13,HLA-DP
4,VSEETGTLIVNSV,0.151,9707.22,70.0,HLA-DPA10301-DPB11301,13,HLA-DP
...,...,...,...,...,...,...,...
125588,KGCCSCGSCCKFDEDDSEPVLKGVK,0.218,4718.05,95.0,HLA-DPA10103-DPB14101,25,HLA-DP
125589,GCCSCGSCCKFDEDDSEPVLKGVKL,0.232,4066.30,95.0,HLA-DPA10103-DPB14101,25,HLA-DP
125590,CCSCGSCCKFDEDDSEPVLKGVKLH,0.248,3417.66,90.0,HLA-DPA10103-DPB14101,25,HLA-DP
125591,CSCGSCCKFDEDDSEPVLKGVKLHY,0.316,1644.14,75.0,HLA-DPA10103-DPB14101,25,HLA-DP


In [11]:
# Write predicted binding affinity predictions.

df = netmhc2_data

df2 = df.groupby(['Peptide', 'loci']).count().reset_index()[['Peptide', 'loci']]
df2['genotype'] = 'unknown'
df2['Rank'] = 0.
df2['1-log50k'] = 0.
df2['nM'] = 0.

data_pivot = pd.concat([df, df2], sort=False).pivot_table(
    index='Peptide',
    columns=['loci', 'genotype'],
    values='1-log50k',
)
data_pivot.to_pickle('mhc2_haplotype_netmhcii-3.2_pred_affinity_pivot_v1v2.pkl.gz', protocol=2)
data_pivot

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT                 0.554     0.364     0.266     0.199     0.091   
AAAYYVGYLQPRTF                0.642     0.456     0.327     0.265     0.108   
AAAYYVGYLQPRTFL               0.659     0.486     0.346     0.297     0.112   
AAAYYVGYLQPRTFLL              0.647     0.460     0.343     0.285     0.113   
AAAYYVGYLQPRTFLLK             0.622     0.432     0.321     0.277     0.109   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM         0.396     0.260     0.201     0.243     0.102   
YYVWKSYVHVVDGCNSSTCMMC        0.397     0.261     0.202     0.246     0.102   
YYVWKSYVHVVDGCNSSTCMMCY       0.398     0.264     0.202     0.248     0.103   
YYVWKSYVHVVDGCNSSTCMMCYK      0.399     0.265     0.203     0.251     0.104   
YYVWKSYVHVVDGCNSSTCMMCYKR     0.402     0.270     0.205     0.255     0.105   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT                 0.403     0.295     0.265     0.364     0.424   
AAAYYVGYLQPRTF                0.458     0.370     0.327     0.443     0.489   
AAAYYVGYLQPRTFL               0.466     0.397     0.357     0.470     0.506   
AAAYYVGYLQPRTFLL              0.467     0.387     0.348     0.457     0.500   
AAAYYVGYLQPRTFLLK             0.445     0.371     0.332     0.442     0.478   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM         0.332     0.259     0.267     0.346     0.372   
YYVWKSYVHVVDGCNSSTCMMC        0.334     0.260     0.269     0.348     0.373   
YYVWKSYVHVVDGCNSSTCMMCY       0.336     0.261     0.271     0.350     0.375   
YYVWKSYVHVVDGCNSSTCMMCYK      0.338     0.263     0.273     0.352     0.376   
YYVWKSYVHVVDGCNSSTCMMCYKR     0.340     0.266     0.278     0.355     0.377   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10309 HLA-DQA10505-DQB10319   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...                 0.327                 0.327   
AAAYYVGYLQPRTF             ...                 0.364                 0.364   
AAAYYVGYLQPRTFL            ...                 0.388                 0.388   
AAAYYVGYLQPRTFLL           ...                 0.396                 0.396   
AAAYYVGYLQPRTFLLK          ...                 0.394                 0.394   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...                 0.392                 0.392   
YYVWKSYVHVVDGCNSSTCMMC     ...                 0.397                 0.397   
YYVWKSYVHVVDGCNSSTCMMCY    ...                 0.408                 0.408   
YYVWKSYVHVVDGCNSSTCMMCYK   ...                 0.413                 0.413   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...                 0.415                 0.415   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10402 HLA-DQA10505-DQB10501   
Peptide                                                                 
AAAYYVGYLQPRT                             0.428                 0.467   
AAAYYVGYLQPRTF                            0.504                 0.524   
AAAYYVGYLQPRTFL                           0.537                 0.559   
AAAYYVGYLQPRTFLL                          0.551                 0.566   
AAAYYVGYLQPRTFLLK                         0.548                 0.558   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

In [12]:
(data_pivot >= 0.638).sum().sum()

174296

In [13]:
# Using % Rank (BA) rather than binding affinity
data_pivot = netmhc2_data.pivot_table(
    index='Peptide',
    columns=['loci', 'genotype'],
    values='Rank',
)
# Invert so binders are >= 98 rather than <= 2.
data_pivot = 100 - data_pivot
data_pivot.to_pickle('mhc2_haplotype_netmhcii-3.2_ba_rank_pivot_v2.pkl.gz', protocol=2)
data_pivot

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT                  79.0      77.0      79.0      40.0      58.0   
AAAYYVGYLQPRTF                 90.0      91.0      88.0      56.0      65.0   
AAAYYVGYLQPRTFL                90.5      93.0      89.0      58.0      60.0   
AAAYYVGYLQPRTFLL               90.0      91.0      90.0      52.0      59.0   
AAAYYVGYLQPRTFLLK              87.0      88.0      87.0      45.0      53.0   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM          30.0      30.0      45.0      25.0      40.0   
YYVWKSYVHVVDGCNSSTCMMC         30.0      30.0      45.0      25.0      40.0   
YYVWKSYVHVVDGCNSSTCMMCY        30.0      30.0      45.0      30.0      40.0   
YYVWKSYVHVVDGCNSSTCMMCYK       30.0      30.0      45.0      30.0      40.0   
YYVWKSYVHVVDGCNSSTCMMCYKR      35.0      30.0      45.0      30.0      40.0   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT                  81.0      69.0      63.0      66.0      85.0   
AAAYYVGYLQPRTF                 87.0      82.0      74.0      78.0      91.5   
AAAYYVGYLQPRTFL                85.0      84.0      75.0      79.0      91.0   
AAAYYVGYLQPRTFLL               86.0      82.0      72.0      76.0      90.5   
AAAYYVGYLQPRTFLLK              82.0      79.0      65.0      72.0      87.0   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM          40.0      35.0      30.0      35.0      54.0   
YYVWKSYVHVVDGCNSSTCMMC         45.0      35.0      30.0      35.0      54.0   
YYVWKSYVHVVDGCNSSTCMMCY        45.0      35.0      35.0      35.0      55.0   
YYVWKSYVHVVDGCNSSTCMMCYK       45.0      35.0      35.0      35.0      56.0   
YYVWKSYVHVVDGCNSSTCMMCYKR      45.0      35.0      35.0      40.0      56.0   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10302 HLA-DQA10505-DQB10309   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...                  83.0                  76.0   
AAAYYVGYLQPRTF             ...                  87.0                  78.0   
AAAYYVGYLQPRTFL            ...                  85.0                  76.0   
AAAYYVGYLQPRTFLL           ...                  81.0                  75.0   
AAAYYVGYLQPRTFLLK          ...                  72.0                  70.0   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...                  40.0                  62.0   
YYVWKSYVHVVDGCNSSTCMMC     ...                  45.0                  63.0   
YYVWKSYVHVVDGCNSSTCMMCY    ...                  51.0                  66.0   
YYVWKSYVHVVDGCNSSTCMMCYK   ...                  52.0                  67.0   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...                  54.0                  67.0   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10319 HLA-DQA10505-DQB10402   
Peptide                                                                 
AAAYYVGYLQPRT                              76.0                  90.0   
AAAYYVGYLQPRTF                             78.0                  96.0   
AAAYYVGYLQPRTFL                            76.0                  96.5   
AAAYYVGYLQPRTFLL                           75.0                  97.0   
AAAYYVGYLQPRTFLLK                          70.0                  96.0   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

In [14]:
(data_pivot >= 98).sum().sum()

626482

In [18]:
(data_pivot >= 90).sum().sum()

3463122

In [17]:
626482 / (125593*280) * 100.

1.781497148953934

# PUFFIN MHC Class II

In [7]:
puffin_mhc2_preds = pd.read_csv('PUFFIN/all_preds_combined_classII.csv.gz')

# Filter dataframe to peptide lengths [13, 25].
puffin_mhc2_preds['sequence_length'] = [len(x) for x in puffin_mhc2_preds['epitope'].values]
puffin_mhc2_preds = puffin_mhc2_preds.loc[puffin_mhc2_preds['sequence_length'].isin(range(13, 26))]

puffin_mhc2_preds.rename(columns={'epitope': 'Peptide', 'mhc': 'genotype'}, inplace=True)

# puffin_mhc2_preds['loci'] = [x[:5] for x in puffin_mhc2_preds['genotype'].values]

puffin_mhc2_preds

,mean_pred,epistemic,aleatoric,binding_likelihood,avg_suff_stat_1,avg_suff_stat_2,genotype,Peptide,sequence_length
0,0.289376,0.003492,0.035966,0.236246,0.289376,0.189648,HLA-DPA10103-DPB10101,MDLFMRIFTIGTVTLKQGEIK,21
1,0.364605,0.006347,0.038386,0.377729,0.364605,0.195923,HLA-DPA10103-DPB10201,MDLFMRIFTIGTVTLKQGEIK,21
2,0.337425,0.006072,0.039994,0.329593,0.337425,0.199984,HLA-DPA10103-DPB10202,MDLFMRIFTIGTVTLKQGEIK,21
3,0.367785,0.004348,0.056473,0.403850,0.367785,0.237641,HLA-DPA10103-DPB10301,MDLFMRIFTIGTVTLKQGEIK,21
4,0.320184,0.005176,0.032980,0.280750,0.320184,0.181603,HLA-DPA10103-DPB10401,MDLFMRIFTIGTVTLKQGEIK,21
...,...,...,...,...,...,...,...,...,...
98967279,0.211966,0.006626,0.028748,0.103811,0.211966,0.169553,HLA-DQA10601-DQB10630,TKLATTEELPDEFVVVTVK,19
98967280,0.230003,0.006200,0.028258,0.122268,0.230003,0.168100,HLA-DQA10601-DQB10632,TKLATTEELPDEFVVVTVK,19
98967281,0.273412,0.004453,0.029170,0.186404,0.273412,0.170792,HLA-DQA10601-DQB10633,TKLATTEELPDEFVVVTVK,19
98967282,0.294119,0.003196,0.028181,0.216706,0.294119,0.167873,HLA-DQA10601-DQB10637,TKLATTEELPDEFVVVTVK,19


In [8]:
puffin_mhc2_new_preds = pd.read_csv('PUFFIN/extra_MHC2_predictions.csv')

# Filter dataframe to peptide lengths [13, 25].
puffin_mhc2_new_preds['sequence_length'] = [len(x) for x in puffin_mhc2_new_preds['epitope'].values]
puffin_mhc2_new_preds = puffin_mhc2_new_preds.loc[puffin_mhc2_new_preds['sequence_length'].isin(range(13, 26))]

puffin_mhc2_new_preds.rename(columns={'epitope': 'Peptide', 'mhc': 'genotype'}, inplace=True)

# puffin_mhc2_new_preds['loci'] = [x[:5] for x in puffin_mhc2_new_preds['genotype'].values]

puffin_mhc2_new_preds

,mean_pred,epistemic,aleatoric,binding_likelihood,avg_suff_stat_1,avg_suff_stat_2,genotype,Peptide,sequence_length
0,0.545357,0.006401,0.046334,0.710976,0.545357,0.215254,DRB1_0101,MDLFMRIFTIGTVT,14
1,0.486121,0.004832,0.044232,0.613191,0.486121,0.210314,DRB1_0102,MDLFMRIFTIGTVT,14
2,0.449011,0.008343,0.042273,0.545279,0.449011,0.205604,DRB1_0103,MDLFMRIFTIGTVT,14
3,0.137176,0.001960,0.018459,0.016875,0.137176,0.135865,DRB1_0301,MDLFMRIFTIGTVT,14
4,0.174542,0.001935,0.023181,0.049561,0.174542,0.152252,DRB1_0302,MDLFMRIFTIGTVT,14
...,...,...,...,...,...,...,...,...,...
20597247,0.370838,0.013723,0.037287,0.388309,0.370838,0.193099,HLA-DQA10505-DQB10501,PIAVQMTKLATTEELPDEFVVVTVK,25
20597248,0.182281,0.003463,0.028081,0.073230,0.182281,0.167575,HLA-DQA10505-DQB10502,PIAVQMTKLATTEELPDEFVVVTVK,25
20597249,0.369109,0.006680,0.022698,0.353784,0.369109,0.150659,HLA-DQA10506-DQB10303,PIAVQMTKLATTEELPDEFVVVTVK,25
20597250,0.244158,0.002368,0.031147,0.151921,0.244158,0.176485,HLA-DQA10508-DQB10301,PIAVQMTKLATTEELPDEFVVVTVK,25


In [9]:
puffin_mhc2_all = pd.concat([puffin_mhc2_preds, puffin_mhc2_new_preds], sort=False)
puffin_mhc2_all

,mean_pred,epistemic,aleatoric,binding_likelihood,avg_suff_stat_1,avg_suff_stat_2,genotype,Peptide,sequence_length
0,0.289376,0.003492,0.035966,0.236246,0.289376,0.189648,HLA-DPA10103-DPB10101,MDLFMRIFTIGTVTLKQGEIK,21
1,0.364605,0.006347,0.038386,0.377729,0.364605,0.195923,HLA-DPA10103-DPB10201,MDLFMRIFTIGTVTLKQGEIK,21
2,0.337425,0.006072,0.039994,0.329593,0.337425,0.199984,HLA-DPA10103-DPB10202,MDLFMRIFTIGTVTLKQGEIK,21
3,0.367785,0.004348,0.056473,0.403850,0.367785,0.237641,HLA-DPA10103-DPB10301,MDLFMRIFTIGTVTLKQGEIK,21
4,0.320184,0.005176,0.032980,0.280750,0.320184,0.181603,HLA-DPA10103-DPB10401,MDLFMRIFTIGTVTLKQGEIK,21
...,...,...,...,...,...,...,...,...,...
20597247,0.370838,0.013723,0.037287,0.388309,0.370838,0.193099,HLA-DQA10505-DQB10501,PIAVQMTKLATTEELPDEFVVVTVK,25
20597248,0.182281,0.003463,0.028081,0.073230,0.182281,0.167575,HLA-DQA10505-DQB10502,PIAVQMTKLATTEELPDEFVVVTVK,25
20597249,0.369109,0.006680,0.022698,0.353784,0.369109,0.150659,HLA-DQA10506-DQB10303,PIAVQMTKLATTEELPDEFVVVTVK,25
20597250,0.244158,0.002368,0.031147,0.151921,0.244158,0.176485,HLA-DQA10508-DQB10301,PIAVQMTKLATTEELPDEFVVVTVK,25


In [ ]:
puffin_mhc2_filtered = puffin_mhc2_all.loc[puffin_mhc2_all['genotype'].isin(set(hla_alleles['allele'].values))]
puffin_mhc2_filtered['loci'] = [x[:4] if x[:3] == 'DRB' else x[:6] for x in puffin_mhc2_filtered['genotype'].values]
puffin_mhc2_filtered

In [18]:
df = puffin_mhc2_filtered

df2 = df.groupby(['Peptide', 'loci']).count().reset_index()[['Peptide', 'loci']]
df2['genotype'] = 'unknown'
df2['mean_pred'] = 0.
df2['binding_likelihood'] = 0.

data_pivot = pd.concat([df, df2], sort=False).pivot_table(
    index='Peptide',
    columns=['loci', 'genotype'],
    values='mean_pred',
)
data_pivot.to_pickle('mhc2_puffin_pred_affinity_pivot_v1v2.pkl.gz', protocol=2)
data_pivot

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT              0.400601  0.312500  0.305490  0.091306  0.135300   
AAAYYVGYLQPRTF             0.580028  0.513865  0.536929  0.160847  0.209960   
AAAYYVGYLQPRTFL            0.648609  0.569664  0.614583  0.189893  0.233327   
AAAYYVGYLQPRTFLL           0.691830  0.570180  0.659630  0.214472  0.273981   
AAAYYVGYLQPRTFLLK          0.728757  0.602119  0.679677  0.255770  0.328044   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM      0.486435  0.325522  0.467445  0.178505  0.250803   
YYVWKSYVHVVDGCNSSTCMMC     0.483641  0.320667  0.466266  0.179284  0.253753   
YYVWKSYVHVVDGCNSSTCMMCY    0.468505  0.317547  0.456332  0.174601  0.253481   
YYVWKSYVHVVDGCNSSTCMMCYK   0.460330  0.309825  0.443855  0.183948  0.260090   
YYVWKSYVHVVDGCNSSTCMMCYKR  0.459879  0.313295  0.437861  0.198431  0.270559   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT              0.289606  0.310504  0.227737  0.257072  0.384428   
AAAYYVGYLQPRTF             0.403355  0.465695  0.351060  0.383100  0.463590   
AAAYYVGYLQPRTFL            0.406949  0.483555  0.369354  0.404608  0.482794   
AAAYYVGYLQPRTFLL           0.443015  0.508008  0.388828  0.416008  0.506840   
AAAYYVGYLQPRTFLLK          0.472579  0.542195  0.418011  0.444002  0.515598   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM      0.409197  0.355279  0.390921  0.414456  0.483765   
YYVWKSYVHVVDGCNSSTCMMC     0.399223  0.350970  0.385805  0.406049  0.472025   
YYVWKSYVHVVDGCNSSTCMMCY    0.381635  0.344578  0.378381  0.396111  0.440015   
YYVWKSYVHVVDGCNSSTCMMCYK   0.378605  0.343438  0.376068  0.395613  0.442305   
YYVWKSYVHVVDGCNSSTCMMCYKR  0.385990  0.355865  0.382627  0.403123  0.450534   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10309 HLA-DQA10505-DQB10319   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...              0.406164              0.406164   
AAAYYVGYLQPRTF             ...              0.437686              0.437686   
AAAYYVGYLQPRTFL            ...              0.419085              0.419085   
AAAYYVGYLQPRTFLL           ...              0.397634              0.397634   
AAAYYVGYLQPRTFLLK          ...              0.389671              0.389671   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...              0.374938              0.374938   
YYVWKSYVHVVDGCNSSTCMMC     ...              0.373139              0.373139   
YYVWKSYVHVVDGCNSSTCMMCY    ...              0.350295              0.350295   
YYVWKSYVHVVDGCNSSTCMMCYK   ...              0.338088              0.338088   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...              0.326963              0.326963   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10402 HLA-DQA10505-DQB10501   
Peptide                                                                 
AAAYYVGYLQPRT                          0.584529              0.456067   
AAAYYVGYLQPRTF                         0.655237              0.542353   
AAAYYVGYLQPRTFL                        0.672601              0.551296   
AAAYYVGYLQPRTFLL                       0.675484              0.540213   
AAAYYVGYLQPRTFLLK                      0.729996              0.531725   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

In [19]:
data_pivot = pd.concat([df, df2], sort=False).pivot_table(
    index='Peptide',
    columns=['loci', 'genotype'],
    values='binding_likelihood',
)
data_pivot.to_pickle('mhc2_puffin_binding_likelihood_pivot_v1v2.pkl.gz', protocol=2)
data_pivot

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT              0.449154  0.275837  0.251057  0.004402  0.019182   
AAAYYVGYLQPRTF             0.763526  0.662420  0.696835  0.043372  0.097140   
AAAYYVGYLQPRTFL            0.830495  0.739159  0.790134  0.095286  0.153303   
AAAYYVGYLQPRTFLL           0.866045  0.723269  0.830479  0.162656  0.245546   
AAAYYVGYLQPRTFLLK          0.899770  0.759244  0.852191  0.232364  0.338678   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM      0.589478  0.342606  0.561539  0.087292  0.197640   
YYVWKSYVHVVDGCNSSTCMMC     0.584971  0.336226  0.559315  0.093154  0.206702   
YYVWKSYVHVVDGCNSSTCMMCY    0.562452  0.333869  0.544634  0.096750  0.213904   
YYVWKSYVHVVDGCNSSTCMMCYK   0.550762  0.322279  0.526595  0.115894  0.223661   
YYVWKSYVHVVDGCNSSTCMMCYKR  0.549986  0.328773  0.517844  0.140979  0.242483   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT              0.209804  0.273397  0.121780  0.160569  0.397723   
AAAYYVGYLQPRTF             0.453153  0.579785  0.343221  0.408718  0.589835   
AAAYYVGYLQPRTFL            0.463464  0.607626  0.387753  0.457538  0.624485   
AAAYYVGYLQPRTFLL           0.529735  0.641095  0.433302  0.482508  0.657231   
AAAYYVGYLQPRTFLLK          0.576275  0.690256  0.486627  0.532303  0.670455   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM      0.473397  0.381563  0.440111  0.480615  0.612662   
YYVWKSYVHVVDGCNSSTCMMC     0.457544  0.376589  0.432581  0.466811  0.588546   
YYVWKSYVHVVDGCNSSTCMMCY    0.430278  0.370461  0.422776  0.451707  0.526737   
YYVWKSYVHVVDGCNSSTCMMCYK   0.424311  0.368985  0.417780  0.450152  0.532259   
YYVWKSYVHVVDGCNSSTCMMCYKR  0.435545  0.389707  0.428690  0.462583  0.548630   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10309 HLA-DQA10505-DQB10319   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...              0.456711              0.456711   
AAAYYVGYLQPRTF             ...              0.526885              0.526885   
AAAYYVGYLQPRTFL            ...              0.485483              0.485483   
AAAYYVGYLQPRTFLL           ...              0.439450              0.439450   
AAAYYVGYLQPRTFLLK          ...              0.423755              0.423755   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...              0.398580              0.398580   
YYVWKSYVHVVDGCNSSTCMMC     ...              0.397415              0.397415   
YYVWKSYVHVVDGCNSSTCMMCY    ...              0.357532              0.357532   
YYVWKSYVHVVDGCNSSTCMMCYK   ...              0.336787              0.336787   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...              0.318870              0.318870   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10402 HLA-DQA10505-DQB10501   
Peptide                                                                 
AAAYYVGYLQPRT                          0.911465              0.568704   
AAAYYVGYLQPRTF                         0.977296              0.753945   
AAAYYVGYLQPRTFL                        0.983811              0.765804   
AAAYYVGYLQPRTFLL                       0.984019              0.734214   
AAAYYVGYLQPRTFLLK                      0.997090              0.715366   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

# Compute ensembles of NetMHCIIpan3.2 and 4.0

In [10]:
netmhc32_aff_pivot = pd.read_pickle(
    'mhc2_haplotype_netmhcii-3.2_pred_affinity_pivot_v1v2.pkl.gz',
)

netmhc40_aff_pivot = pd.read_pickle(
    'mhc2_haplotype_netmhcii-4.0_pred_affinity_pivot_v1v2.pkl.gz',
)

netmhc32_ba_rank_pivot = pd.read_pickle(
    'mhc2_haplotype_netmhcii-3.2_ba_rank_pivot_v2.pkl.gz',
)

netmhc40_el_rank_pivot = pd.read_pickle(
    'mhc2_haplotype_netmhcii-4.0_el_rank_pivot_v2.pkl.gz',
)

netmhc40_ba_rank_pivot = pd.read_pickle(
    'mhc2_haplotype_netmhcii-4.0_ba_rank_pivot_v2.pkl.gz',
)

In [13]:
# Average of 3.2 BA %rank and 4.0 EL %rank
ens_32ba_40el = (netmhc32_ba_rank_pivot + netmhc40_el_rank_pivot) / 2
ens_32ba_40el.to_pickle('mhc2_haplotype_ens_netmhcii-3.2_ba_rank_netmhcii-4.0_el_rank.pkl.gz')
ens_32ba_40el

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT                64.275    59.650    68.835    51.565    65.980   
AAAYYVGYLQPRTF               73.455    75.120    78.685    59.470    70.130   
AAAYYVGYLQPRTFL              74.710    83.220    80.270    58.335    66.605   
AAAYYVGYLQPRTFLL             70.505    79.030    77.195    51.385    59.000   
AAAYYVGYLQPRTFLLK            78.045    81.340    80.755    43.050    51.195   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM        17.675    17.500    27.870    15.120    24.100   
YYVWKSYVHVVDGCNSSTCMMC       17.890    17.500    28.265    15.065    24.250   
YYVWKSYVHVVDGCNSSTCMMCY      18.135    17.710    28.690    17.710    24.400   
YYVWKSYVHVVDGCNSSTCMMCYK     18.365    17.910    29.115    17.840    24.500   
YYVWKSYVHVVDGCNSSTCMMCYKR    21.085    18.135    29.395    17.935    24.650   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT                64.140    61.215    55.335    48.245    65.205   
AAAYYVGYLQPRTF               67.860    75.370    62.330    58.140    73.335   
AAAYYVGYLQPRTFL              62.605    74.825    58.555    54.250    69.085   
AAAYYVGYLQPRTFLL             56.910    67.635    51.110    48.885    64.000   
AAAYYVGYLQPRTFLLK            62.300    62.605    47.870    47.880    60.040   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM        40.645    26.405    27.935    29.590    55.095   
YYVWKSYVHVVDGCNSSTCMMC       43.865    26.715    28.455    30.230    55.515   
YYVWKSYVHVVDGCNSSTCMMCY      44.470    27.145    31.410    30.680    56.340   
YYVWKSYVHVVDGCNSSTCMMCYK     44.980    27.505    31.800    31.060    57.095   
YYVWKSYVHVVDGCNSSTCMMCYKR    45.310    27.755    32.130    33.945    57.305   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10302 HLA-DQA10505-DQB10309   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...                61.845                80.165   
AAAYYVGYLQPRTF             ...                68.745                80.400   
AAAYYVGYLQPRTFL            ...                63.005                75.580   
AAAYYVGYLQPRTFLL           ...                58.000                69.810   
AAAYYVGYLQPRTFLLK          ...                51.745                61.085   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...                23.240                33.535   
YYVWKSYVHVVDGCNSSTCMMC     ...                25.690                34.015   
YYVWKSYVHVVDGCNSSTCMMCY    ...                28.915                35.615   
YYVWKSYVHVVDGCNSSTCMMCYK   ...                29.555                36.445   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...                30.730                38.105   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10319 HLA-DQA10505-DQB10402   
Peptide                                                                 
AAAYYVGYLQPRT                            80.165                81.165   
AAAYYVGYLQPRTF                           80.400                87.275   
AAAYYVGYLQPRTFL                          75.580                82.425   
AAAYYVGYLQPRTFLL                         69.810                74.615   
AAAYYVGYLQPRTFLLK                        61.085                67.705   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

In [14]:
# Average of 4.0 BA %rank and 4.0 EL %rank
ens_40ba_40el = (netmhc40_ba_rank_pivot + netmhc40_el_rank_pivot) / 2
ens_40ba_40el.to_pickle('mhc2_haplotype_ens_netmhcii-4.0_ba_rank_netmhcii-4.0_el_rank.pkl.gz')
ens_40ba_40el

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT                40.410    34.845    47.735    45.705    54.070   
AAAYYVGYLQPRTF               62.335    64.590    68.555    56.265    65.645   
AAAYYVGYLQPRTFL              75.720    84.395    82.140    68.555    78.895   
AAAYYVGYLQPRTFLL             71.065    78.640    76.480    59.335    68.725   
AAAYYVGYLQPRTFLLK            82.395    84.575    84.650    59.635    67.115   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM        33.125    25.665    33.590    27.885    38.615   
YYVWKSYVHVVDGCNSSTCMMC       33.475    26.320    34.675    28.575    39.010   
YYVWKSYVHVVDGCNSSTCMMCY      33.910    27.275    35.650    29.345    39.790   
YYVWKSYVHVVDGCNSSTCMMCYK     34.375    28.315    36.565    30.125    40.160   
YYVWKSYVHVVDGCNSSTCMMCYKR    34.980    29.515    37.400    30.905    40.690   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT                38.715    48.080    39.010    29.135    42.805   
AAAYYVGYLQPRTF               53.500    69.305    53.950    47.765    62.370   
AAAYYVGYLQPRTFL              61.415    77.655    61.650    56.285    68.545   
AAAYYVGYLQPRTFLL             54.890    67.390    53.540    49.145    63.180   
AAAYYVGYLQPRTFLLK            65.950    66.410    57.615    54.085    62.780   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM        59.115    37.935    46.760    45.895    71.040   
YYVWKSYVHVVDGCNSSTCMMC       60.160    38.905    47.475    46.925    71.555   
YYVWKSYVHVVDGCNSSTCMMCY      61.000    39.695    48.140    47.720    72.000   
YYVWKSYVHVVDGCNSSTCMMCYK     61.920    40.360    48.870    48.435    72.340   
YYVWKSYVHVVDGCNSSTCMMCYKR    62.580    40.980    49.620    49.140    72.630   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10302 HLA-DQA10505-DQB10309   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...                39.830                64.315   
AAAYYVGYLQPRTF             ...                62.370                75.925   
AAAYYVGYLQPRTFL            ...                61.085                77.150   
AAAYYVGYLQPRTFLL           ...                57.965                71.060   
AAAYYVGYLQPRTFLLK          ...                51.710                62.945   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...                25.230                30.695   
YYVWKSYVHVVDGCNSSTCMMC     ...                25.800                31.645   
YYVWKSYVHVVDGCNSSTCMMCY    ...                29.690                33.640   
YYVWKSYVHVVDGCNSSTCMMCYK   ...                30.425                35.785   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...                31.490                39.195   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10319 HLA-DQA10505-DQB10402   
Peptide                                                                 
AAAYYVGYLQPRT                            64.315                61.720   
AAAYYVGYLQPRTF                           75.925                83.850   
AAAYYVGYLQPRTFL                          77.150                82.920   
AAAYYVGYLQPRTFLL                         71.060                74.060   
AAAYYVGYLQPRTFLLK                        62.945                67.310   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

In [15]:
# Average of 3.2 predicted affinity and 4.0 predicted affinity
ens_32aff_40aff = 1 - np.log(((50000 ** (1-netmhc32_aff_pivot)) + (50000 ** (1-netmhc40_aff_pivot))) / 2) / np.log(50000)
ens_32aff_40aff.to_pickle('mhc2_haplotype_ens_netmhcii-3.2_aff_netmhcii-4.0_aff.pkl.gz')
ens_32aff_40aff

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT              0.322484  0.258498  0.200212  0.163577  0.076705   
AAAYYVGYLQPRTF             0.490004  0.394190  0.267980  0.219368  0.093999   
AAAYYVGYLQPRTFL            0.637906  0.506652  0.348317  0.288839  0.115880   
AAAYYVGYLQPRTFLL           0.622981  0.465672  0.329097  0.259959  0.110437   
AAAYYVGYLQPRTFLLK          0.642850  0.465804  0.344080  0.278532  0.114585   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM      0.397788  0.266286  0.202335  0.213601  0.098355   
YYVWKSYVHVVDGCNSSTCMMC     0.398960  0.268777  0.204457  0.217067  0.098649   
YYVWKSYVHVVDGCNSSTCMMCY    0.400411  0.272610  0.205723  0.219780  0.099916   
YYVWKSYVHVVDGCNSSTCMMCYK   0.402106  0.275694  0.207381  0.223090  0.100753   
YYVWKSYVHVVDGCNSSTCMMCYKR  0.405549  0.281402  0.209714  0.226775  0.101743   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT              0.254549  0.222623  0.192972  0.246155  0.289692   
AAAYYVGYLQPRTF             0.339910  0.290397  0.250138  0.348641  0.388828   
AAAYYVGYLQPRTFL            0.419452  0.355387  0.305167  0.441926  0.473647   
AAAYYVGYLQPRTFLL           0.417310  0.324221  0.292519  0.413951  0.466066   
AAAYYVGYLQPRTFLLK          0.439120  0.335532  0.305352  0.435093  0.475456   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM      0.345216  0.235960  0.248369  0.345126  0.393548   
YYVWKSYVHVVDGCNSSTCMMC     0.347618  0.238340  0.249769  0.347843  0.394617   
YYVWKSYVHVVDGCNSSTCMMCY    0.349694  0.239857  0.251226  0.350417  0.396423   
YYVWKSYVHVVDGCNSSTCMMCYK   0.352444  0.241563  0.253076  0.352767  0.397452   
YYVWKSYVHVVDGCNSSTCMMCYKR  0.354909  0.243884  0.256366  0.355690  0.398470   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10309 HLA-DQA10505-DQB10319   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...              0.280495              0.280495   
AAAYYVGYLQPRTF             ...              0.347394              0.347394   
AAAYYVGYLQPRTFL            ...              0.385707              0.385707   
AAAYYVGYLQPRTFLL           ...              0.384422              0.384422   
AAAYYVGYLQPRTFLLK          ...              0.372327              0.372327   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...              0.324329              0.324329   
YYVWKSYVHVVDGCNSSTCMMC     ...              0.330587              0.330587   
YYVWKSYVHVVDGCNSSTCMMCY    ...              0.343593              0.343593   
YYVWKSYVHVVDGCNSSTCMMCYK   ...              0.354990              0.354990   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...              0.365810              0.365810   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10402 HLA-DQA10505-DQB10501   
Peptide                                                                 
AAAYYVGYLQPRT                          0.341747              0.319118   
AAAYYVGYLQPRTF                         0.463792              0.391967   
AAAYYVGYLQPRTFL                        0.524090              0.435281   
AAAYYVGYLQPRTFLL                       0.515780              0.426203   
AAAYYVGYLQPRTFLLK                      0.509578              0.430653   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

In [16]:
netmhc32_aff_pivot

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT                 0.554     0.364     0.266     0.199     0.091   
AAAYYVGYLQPRTF                0.642     0.456     0.327     0.265     0.108   
AAAYYVGYLQPRTFL               0.659     0.486     0.346     0.297     0.112   
AAAYYVGYLQPRTFLL              0.647     0.460     0.343     0.285     0.113   
AAAYYVGYLQPRTFLLK             0.622     0.432     0.321     0.277     0.109   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM         0.396     0.260     0.201     0.243     0.102   
YYVWKSYVHVVDGCNSSTCMMC        0.397     0.261     0.202     0.246     0.102   
YYVWKSYVHVVDGCNSSTCMMCY       0.398     0.264     0.202     0.248     0.103   
YYVWKSYVHVVDGCNSSTCMMCYK      0.399     0.265     0.203     0.251     0.104   
YYVWKSYVHVVDGCNSSTCMMCYKR     0.402     0.270     0.205     0.255     0.105   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT                 0.403     0.295     0.265     0.364     0.424   
AAAYYVGYLQPRTF                0.458     0.370     0.327     0.443     0.489   
AAAYYVGYLQPRTFL               0.466     0.397     0.357     0.470     0.506   
AAAYYVGYLQPRTFLL              0.467     0.387     0.348     0.457     0.500   
AAAYYVGYLQPRTFLLK             0.445     0.371     0.332     0.442     0.478   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM         0.332     0.259     0.267     0.346     0.372   
YYVWKSYVHVVDGCNSSTCMMC        0.334     0.260     0.269     0.348     0.373   
YYVWKSYVHVVDGCNSSTCMMCY       0.336     0.261     0.271     0.350     0.375   
YYVWKSYVHVVDGCNSSTCMMCYK      0.338     0.263     0.273     0.352     0.376   
YYVWKSYVHVVDGCNSSTCMMCYKR     0.340     0.266     0.278     0.355     0.377   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10309 HLA-DQA10505-DQB10319   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...                 0.327                 0.327   
AAAYYVGYLQPRTF             ...                 0.364                 0.364   
AAAYYVGYLQPRTFL            ...                 0.388                 0.388   
AAAYYVGYLQPRTFLL           ...                 0.396                 0.396   
AAAYYVGYLQPRTFLLK          ...                 0.394                 0.394   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...                 0.392                 0.392   
YYVWKSYVHVVDGCNSSTCMMC     ...                 0.397                 0.397   
YYVWKSYVHVVDGCNSSTCMMCY    ...                 0.408                 0.408   
YYVWKSYVHVVDGCNSSTCMMCYK   ...                 0.413                 0.413   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...                 0.415                 0.415   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10402 HLA-DQA10505-DQB10501   
Peptide                                                                 
AAAYYVGYLQPRT                             0.428                 0.467   
AAAYYVGYLQPRTF                            0.504                 0.524   
AAAYYVGYLQPRTFL                           0.537                 0.559   
AAAYYVGYLQPRTFLL                          0.551                 0.566   
AAAYYVGYLQPRTFLLK                         0.548                 0.558   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               

In [17]:
netmhc40_aff_pivot

loci                           DRB1                                          \
genotype                  DRB1_0101 DRB1_0102 DRB1_0103 DRB1_0301 DRB1_0302   
Peptide                                                                       
AAAYYVGYLQPRT              0.262275  0.210513  0.162170  0.138032  0.064328   
AAAYYVGYLQPRTF             0.435325  0.357479  0.232249  0.188956  0.081843   
AAAYYVGYLQPRTFL            0.620743  0.533288  0.350694  0.281340  0.119930   
AAAYYVGYLQPRTFLL           0.603934  0.471715  0.317015  0.240276  0.107944   
AAAYYVGYLQPRTFLLK          0.669816  0.519657  0.374913  0.280090  0.120530   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM      0.399611  0.273032  0.203689  0.191331  0.094848   
YYVWKSYVHVVDGCNSSTCMMC     0.400962  0.277270  0.206981  0.195065  0.095416   
YYVWKSYVHVVDGCNSSTCMMCY    0.402886  0.282106  0.209602  0.198191  0.096931   
YYVWKSYVHVVDGCNSSTCMMCYK   0.405321  0.287790  0.211981  0.201682  0.097617   
YYVWKSYVHVVDGCNSSTCMMCYKR  0.409240  0.294411  0.214682  0.205183  0.098597   

loci                                                                         \
genotype                  DRB1_0401 DRB1_0402 DRB1_0403 DRB1_0404 DRB1_0405   
Peptide                                                                       
AAAYYVGYLQPRT              0.200257  0.182535  0.152988  0.196000  0.237120   
AAAYYVGYLQPRTF             0.289715  0.248274  0.208763  0.302934  0.341891   
AAAYYVGYLQPRTFL            0.388640  0.326797  0.272157  0.420422  0.449729   
AAAYYVGYLQPRTFLL           0.385170  0.287178  0.258092  0.384697  0.441300   
AAAYYVGYLQPRTFLLK          0.433591  0.309963  0.284692  0.428667  0.472981   
...                             ...       ...       ...       ...       ...   
YYVWKSYVHVVDGCNSSTCMM      0.360642  0.217532  0.232871  0.344261  0.421699   
YYVWKSYVHVVDGCNSSTCMMC     0.363595  0.220804  0.233858  0.347686  0.422885   
YYVWKSYVHVVDGCNSSTCMMCY    0.365776  0.222661  0.234946  0.350836  0.424360   
YYVWKSYVHVVDGCNSSTCMMCYK   0.369570  0.224174  0.236694  0.353541  0.425438   
YYVWKSYVHVVDGCNSSTCMMCYKR  0.372695  0.226051  0.238848  0.356386  0.426486   

loci                       ...                HLA-DQ                        \
genotype                   ... HLA-DQA10505-DQB10309 HLA-DQA10505-DQB10319   
Peptide                    ...                                               
AAAYYVGYLQPRT              ...              0.249702              0.249702   
AAAYYVGYLQPRTF             ...              0.333322              0.333322   
AAAYYVGYLQPRTFL            ...              0.383470              0.383470   
AAAYYVGYLQPRTFLL           ...              0.374134              0.374134   
AAAYYVGYLQPRTFLLK          ...              0.354784              0.354784   
...                        ...                   ...                   ...   
YYVWKSYVHVVDGCNSSTCMM      ...              0.285683              0.285683   
YYVWKSYVHVVDGCNSSTCMMC     ...              0.292342              0.292342   
YYVWKSYVHVVDGCNSSTCMMCY    ...              0.306004              0.306004   
YYVWKSYVHVVDGCNSSTCMMCYK   ...              0.319624              0.319624   
YYVWKSYVHVVDGCNSSTCMMCYKR  ...              0.333877              0.333877   

loci                                                                   \
genotype                  HLA-DQA10505-DQB10402 HLA-DQA10505-DQB10501   
Peptide                                                                 
AAAYYVGYLQPRT                          0.297920              0.264890   
AAAYYVGYLQPRTF                         0.435867              0.339701   
AAAYYVGYLQPRTFL                        0.512765              0.384206   
AAAYYVGYLQPRTFLL                       0.490339              0.372928   
AAAYYVGYLQPRTFLLK                      0.482520              0.379043   
...                                         ...                   ...   
YYVWKSYVHVVDGCNSSTCMM               